In [ ]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

In [ ]:
# Visit the AdvancedStats to Scrape site
url = 'https://www.pro-football-reference.com/years/2018/passing_advanced.htm'
browser.visit(url)

In [ ]:
# Parse the HTML
html = browser.html
html_soup = soup(html, 'html.parser')

In [ ]:
adv_ay = html_soup.find('table', {"id" : 'advanced_air_yards'})
adv_acc = html_soup.find('table', {"id" : 'advanced_accuracy'})
adv_pres = html_soup.find('table', {"id" : 'advanced_pressure'})
adv_pt = html_soup.find('table', {"id" : 'advanced_play_type'})

In [ ]:
import pandas as pd
advanced_air_yard = pd.read_html(str(adv_ay),header=1)[0]
advanced_accuracy = pd.read_html(str(adv_acc),header=1)[0]
advanced_pressure = pd.read_html(str(adv_pres),header=1)[0]
advanced_play_type = pd.read_html(str(adv_pt),header=1)[0]

In [ ]:
from functools import reduce
pd.set_option('display.max_rows', None)
cols_to_use = advanced_accuracy.columns.difference(advanced_air_yard.columns)
data_frames = [advanced_air_yard, advanced_accuracy, advanced_pressure, advanced_play_type]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                            how='outer'), data_frames)
df_merged = df_merged[df_merged['Player'] != 'Player']
df_merged = df_merged[df_merged['Player'].notna()]
df_merged = df_merged.replace('^qb','QB',regex=True).astype(object)
df_merged = df_merged.replace('^rb','RB',regex=True).astype(object)
df_merged = df_merged.replace('^wr','WR',regex=True).astype(object)
df_merged = df_merged.replace('^te','TE',regex=True).astype(object)
df_merged = df_merged.replace('\+','',regex=True).astype(object)
df_merged = df_merged.replace('\*','',regex=True).astype(object)
columns = ['Player','Tm_x','Pos_x','Age_x','G_x','GS_x','Cmp_x','Att_x','Yds_x','IAY',
           'IAY/PA','CAY','CAY/Cmp','CAY/PA','YAC','YAC/Cmp','Bats','ThAwy',
           'Spikes','Drops','Drop%','BadTh','Bad%','OnTgt','OnTgt%','Sk',
           'PktTime','Bltz','Hrry','Prss','Prss%','Scrm','Yds/Scr','Plays',
           'Yds.1','PassAtt','PassYds','RushAtt','RushYds', 'PassAtt.1', 'PassYds.1']

df_merged = df_merged[columns]
df_merged = df_merged.loc[:,~df_merged.columns.duplicated()]
cols = []
for column in df_merged.columns:
    if column == 'Tm_x':
        cols.append('Tm')
    elif column == 'Pos_x':
        cols.append('Pos')
    elif column == 'Age_x':
        cols.append('Age')
    elif column == 'G_x':
        cols.append('G')
    elif column == 'GS_x':
        cols.append('GS')
    elif column == 'Cmp_x':
        cols.append('Cmp')
    elif column == 'Att_x':
        cols.append('Att')
    elif column == 'Yds_x':
        cols.append('Yds')
    elif column == 'Yds.1':
        cols.append('RPO_Yds')
    elif column == 'PassAtt':
        cols.append('RPO_PassAtt')
    elif column == 'PassYds':
        cols.append('RPO_PassYds')
    elif column == 'RushAtt':
        cols.append('RPO_RushAtt')
    elif column == 'RushYds':
        cols.append('RPO_RushYds')
    elif column == 'PassAtt.1':
        cols.append('PA_PassAtt')
    elif column == 'PassYds.1':
        cols.append('PA_PassYds')
    else:
        cols.append(column)

df_merged.columns = cols
df_merged = df_merged.fillna(0)
df_merged = df_merged[df_merged['Pos'] != 'k']
df_merged = df_merged[df_merged['Pos'] != 'p']
columns = df_merged.columns[3:]
for col in columns:
    if '%' in col:
        df_merged[col] = df_merged[col].str.rstrip('%').astype('float') / 100.0
    else:
        df_merged[col] = pd.to_numeric(df_merged[col],errors='coerce')
df_merged

In [ ]:
df_merged.columns

In [ ]:
df_merged.dtypes

In [ ]:
df_merged['Bad%']

In [ ]:
df_merged['Pos'].value_counts()

In [ ]:
df_merged.isnull().sum()

In [ ]:
df_merged[df_merged['Pos'].isnull()]

In [ ]:
browser.quit()

In [ ]:
def scrapeAdvancedPass(year):
    from splinter import Browser
    from bs4 import BeautifulSoup as soup
    from webdriver_manager.chrome import ChromeDriverManager
    import pandas as pd
    from functools import reduce
    # Set up Splinter
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=True)
    # Visit the AdvancedStats to Scrape site
    url = f'https://www.pro-football-reference.com/years/{year}/passing_advanced.htm'
    browser.visit(url)
    # Parse the HTML
    html = browser.html
    html_soup = soup(html, 'html.parser')
    try:
        
        df1 = html_soup.find('table', {"id" : 'advanced_air_yards'})
        df2 = html_soup.find('table', {"id" : 'advanced_accuracy'})
        df3 = html_soup.find('table', {"id" : 'advanced_pressure'})
        df4 = html_soup.find('table', {"id" : 'advanced_play_type'})

        df1 = pd.read_html(str(df1),header=1)[0]
        df2 = pd.read_html(str(df2),header=1)[0]
        df3 = pd.read_html(str(df3),header=1)[0]
        df4 = pd.read_html(str(df4),header=1)[0]
    
        data_frames = [df1, df2, df3, df4]
        df = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                                    how='outer'), data_frames)
        df = df[df['Player'] != 'Player']
        df = df[df['Player'].notna()]
        df = df.replace('^qb','QB',regex=True).astype(object)
        df = df.replace('^rb','RB',regex=True).astype(object)
        df = df.replace('^wr','WR',regex=True).astype(object)
        df = df.replace('^te','TE',regex=True).astype(object)
        df = df.replace('\+','',regex=True).astype(object)
        df = df.replace('\*','',regex=True).astype(object)
        columns = ['Player','Tm_x','Pos_x','Age_x','G_x','GS_x','Cmp_x','Att_x','Yds_x','IAY',
                   'IAY/PA','CAY','CAY/Cmp','CAY/PA','YAC','YAC/Cmp','Bats','ThAwy',
                   'Spikes','Drops','Drop%','BadTh','Bad%','OnTgt','OnTgt%','Sk',
                   'PktTime','Bltz','Hrry','Prss','Prss%','Scrm','Yds/Scr','Plays',
                   'Yds.1','PassAtt','PassYds','RushAtt','RushYds', 'PassAtt.1', 'PassYds.1']

        df = df[columns]
        df = df.loc[:,~df.columns.duplicated()]
        cols = []
        for column in df.columns:
            if column == 'Tm_x':
                cols.append('Tm')
            elif column == 'Pos_x':
                cols.append('Pos')
            elif column == 'Age_x':
                cols.append('Age')
            elif column == 'G_x':
                cols.append('G')
            elif column == 'GS_x':
                cols.append('GS')
            elif column == 'Cmp_x':
                cols.append('Cmp')
            elif column == 'Att_x':
                cols.append('Att')
            elif column == 'Yds_x':
                cols.append('Yds')
            elif column == 'Yds.1':
                cols.append('RPO_Yds')
            elif column == 'PassAtt':
                cols.append('RPO_PassAtt')
            elif column == 'PassYds':
                cols.append('RPO_PassYds')
            elif column == 'RushAtt':
                cols.append('RPO_RushAtt')
            elif column == 'RushYds':
                cols.append('RPO_RushYds')
            elif column == 'PassAtt.1':
                cols.append('PA_PassAtt')
            elif column == 'PassYds.1':
                cols.append('PA_PassYds')
            else:
                cols.append(column)

        df.columns = cols
        df = df.fillna(0)
        df = df[df['Pos'] != 'k']
        df = df[df['Pos'] != 'p']
        columns = df.columns[3:]
        for col in columns:
            if '%' in col:
                df[col] = df[col].str.rstrip('%').astype('float') / 100.0
            else:
                df[col] = pd.to_numeric(df[col],errors='coerce')
    except:
        
        df1 = html_soup.find('table', {"id" : 'advanced_air_yards'})
        df2 = html_soup.find('table', {"id" : 'advanced_accuracy'})
        df3 = html_soup.find('table', {"id" : 'advanced_pressure'})

        df1 = pd.read_html(str(df1),header=1)[0]
        df2 = pd.read_html(str(df2),header=1)[0]
        df3 = pd.read_html(str(df3),header=1)[0]
    
        data_frames = [df1, df2, df3]
        df = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
                                                    how='outer'), data_frames)
        df = df[df['Player'] != 'Player']
        df = df[df['Player'].notna()]
        df = df.replace('^qb','QB',regex=True).astype(object)
        df = df.replace('^rb','RB',regex=True).astype(object)
        df = df.replace('^wr','WR',regex=True).astype(object)
        df = df.replace('^te','TE',regex=True).astype(object)
        df = df.replace('\+','',regex=True).astype(object)
        df = df.replace('\*','',regex=True).astype(object)
        columns = ['Player','Tm_x','Pos_x','Age_x','G_x','GS_x','Cmp_x','Att_x','Yds_x','IAY',
                   'IAY/PA','CAY','CAY/Cmp','CAY/PA','YAC','YAC/Cmp','ThAwy',
                   'Spikes','Drops','Drop%','BadTh','Bad%','Sk',
                   'PktTime','Bltz','Hrry','Prss','Prss%','Scrm','Yds/Scr']

        df = df[columns]
        df = df.loc[:,~df.columns.duplicated()]
        cols = []
        for column in df.columns:
            if column == 'Tm_x':
                cols.append('Tm')
            elif column == 'Pos_x':
                cols.append('Pos')
            elif column == 'Age_x':
                cols.append('Age')
            elif column == 'G_x':
                cols.append('G')
            elif column == 'GS_x':
                cols.append('GS')
            elif column == 'Cmp_x':
                cols.append('Cmp')
            elif column == 'Att_x':
                cols.append('Att')
            elif column == 'Yds_x':
                cols.append('Yds')
            else:
                cols.append(column)

        df.columns = cols
        df = df.fillna(0)
        df = df[df['Pos'] != 'k']
        df = df[df['Pos'] != 'p']
        columns = df.columns[3:]
        for col in columns:
            if '%' in col:
                df[col] = df[col].str.rstrip('%').astype('float') / 100.0
            else:
                df[col] = pd.to_numeric(df[col],errors='coerce')
    
    browser.quit()
    
    return df